In [11]:
# Importing dependecies:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
from collections import Counter

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

import matplotlib.pyplot as plt
import seaborn as sns
plt.switch_backend('agg')
%matplotlib inline
from pandas import compat
compat.PY3 = True

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv('data/strings/df_clean.csv', index_col=0)
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b..."
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak..."
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',..."
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook..."


## Word2Vec:

In [ ]:
# model_ingredients = gs.Word2Vec(df['ingredients_vector'], min_count=1, size= 25, workers=3, window=3, sg=1)
# model_instructions = gs.Word2Vec(df['instructions_vector'], min_count=1, size= 25, workers=3, window=3, sg=1)

In [ ]:
# print(model_ingredients)
# data = model_instructions.most_similar('beef')
# print(data)

## Encoding Text:

In [14]:
X = df['ingredients']
y = df['title']
print(X[:5])

p3pKOD6jIHEcjf20CCXohP8uqkG5dGi    celery finely chopped green pepper finely chop...
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO    skirt steak cut inch dicekosher salt fresh cra...
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK    cups dried black beans picked rinsed cups wate...
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK    ground chuckone tomato sauce saltfreshly groun...
kRBQSWtqYWqtkb34FGeenBSbC32gIdO    rice brown mediumgrain cookedcup quinoacup swe...
Name: ingredients, dtype: object


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [5]:
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])

In [6]:
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])

In [7]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [8]:
# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [15]:
# create tokenizer
tokenizer = create_tokenizer(X)
# calculate max document length
length = max_length(X)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, y, length)
print(trainX.shape)

Max document length: 338
Vocabulary size: 45441
(59612, 338)


In [ ]:
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], np.array(y), epochs=10, batch_size=16)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 338)          0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 338)          0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 338)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 338, 100)     4544100     input_10[0][0]                   
__________________________________________________________________________________________________
embedding_

In [ ]:
vocab_size = 1000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(X_train)

In [ ]:
X_train = tokenize.texts_to_matrix(X_train)

In [ ]:
encoder = preprocessing.LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
X_train.shape, y_test.shape, X_test.shape, y_test.shape

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', 
          optimizer='adam', 
          metrics=['accuracy'])
history = model.fit(X_train, y_train, 
                    batch_size=100, 
                    epochs=2, 
                    verbose=1, 
                    validation_split=0.1)